In [ ]:
#drive = True
drive = False

In [ ]:
if drive:
  !pip install tensorflow-gpu==2.6
  #!pip install keras==2.6 -q
  !pip install scikit-learn==0.22.2.post1 -q
  !pip install scikit-image==0.19.0 -q
  !pip install numpy==1.21.2 -q


import sklearn
print('\nsklearn:', sklearn.__version__)
import skimage
print('skimage:', skimage.__version__)
import cv2
print('cv2:', cv2.__version__)
import tensorflow as tf
print('tf:', tf.__version__)
import keras
print('keras:', keras.__version__)


In [ ]:
if drive:
  from google.colab import drive
  drive.mount('/content/gdrive/', force_remount=True)
  from os.path import dirname as up
  %cd /content/gdrive/My Drive/Research/pelagicas/Codigos Diego Ramirez/Otros/Mas Codigos/Python/Modelos en Keras v3 (paper)/_FishNet

import pickle
import copy
import numpy as np
import os
from collections import defaultdict
from sklearn.model_selection import StratifiedKFold, KFold
#from tensorflow.keras.models import load_model
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import tensorflow as tf
import time
from collections import Counter
from sklearn.metrics import zero_one_loss, accuracy_score, precision_score, recall_score, f1_score
from matplotlib import pylab
from collections import Counter
import gc
seed = 42

In [ ]:
#Select classifier
#featureCLF = 'DL-RGB'
#featureCLF = 'DL-SPEC'
featureCLF = 'DL-BOTH'
#featureCLF = 'IE'
#featureCLF = 'HOG'

In [ ]:
if 'DL' in featureCLF:
  from DL import *
else:
  from ML import *
  from DL import getDataPath, getClasses
from utils import *

In [ ]:
if drive:
  mainFolder = './/data' 
  trainFolder = mainFolder + '//DataTrainVal'
else:
  mainFolder = '.\\data'
  trainFolder = mainFolder + '\\DataTrainVal'

trainPath = getDataPath(trainFolder)

In [ ]:
if 'HOG' in featureCLF:
  data, classes = LoadHOGFeatures(trainPath)
else:
  data, classes = LoadDataFish(trainPath)

total_samples = classes.shape[0]
print(total_samples)
print(np.sum(classes))

In [ ]:
if 'DL' in featureCLF:

  Labels = ['Anchoveta', 'Merluza', 'Mote', 'Pampanito', 'Sardina Comun']

  learning_rate = 0.05
  decay = 1e-3
  momentum = 0.9
  optimizer = SGD(lr = learning_rate)#,  decay = decay, momentum = momentum
  optimizer_params = {'learning_rate':learning_rate, 'decay':decay, 'momentum':momentum}

  dropProbability = 0.25
  activationUse = 'relu'
  num_classes = 5
  kinit = tf.keras.initializers.RandomUniform(seed=seed)
  kernel_initializer = [kinit, kinit]
  n_filters = np.asarray([[32,16],
                          [30,10]])
  kernel_size = np.asarray([ [(2,2),(2,2)],
                            [3,3]])
  pool_size = np.asarray([[(2,2),(2,2)],
                          [2,2]])
  dense = np.asarray([125,
                      75,
                      25])
  model_params = {'kernel_initializer':kernel_initializer, 'n_filters':n_filters, 'kernel_size':kernel_size, 'pool_size':pool_size, 'dropProbability':dropProbability, 'activationUse':activationUse, 'dense':dense}

  model = defineModelo(model_params, featureCLF)

  validation_split = 0.33
  batch_size = 32

  if not drive:
    savePath = '.\\MejorModeloAllNew\\'
  else:
    savePath = os.path.join( os.getcwd(),  'MejorModeloAllNew' )
  if(dropProbability > 0):
      if not drive:
        savePath += 'ConDrop'+str(int(dropProbability*100))+'_\\'
      else:
        savePath += 'ConDrop'+str(int(dropProbability*100))+'_//'
  else:
      if not drive:
        savePath += 'SinDrop\\'
      else:
        savePath += 'SinDrop//'

  if not os.path.exists(savePath):
      if not drive:
        savePath += '0-'+activationUse+'\\'
      else:
        savePath += '0-'+activationUse+'//'
      os.makedirs(savePath)
  else:
      dirPath = next(os.walk(savePath))[1]
      if not drive:
        savePath += str(len(dirPath))+'-'+activationUse+'\\'
      else:
        savePath += str(len(dirPath))+'-'+activationUse+'//'
      os.makedirs(savePath)

  model.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics=["acc"])
  plot_model(model, to_file= os.path.join( os.getcwd(), 'out', 'model', 'FishNet_plot.png' ) , show_shapes=True, show_layer_names=True)

In [ ]:
if 'DL' in featureCLF:
  Epochs = 4000
  filepath = savePath + 'Model_2Input'
  Maximo = False

  if (Maximo):
      checkpoint = ModelCheckpoint(filepath+'.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
      earlystop = EarlyStopping(monitor='val_acc', patience=50, mode='max')
  else:
      checkpoint = ModelCheckpoint( os.path.join(os.getcwd(), 'out','model', 'Model_2Input.hdf5'), monitor='val_loss', verbose=0, save_best_only=True, mode='min')
      earlystop = EarlyStopping(monitor='val_loss', patience=50, mode='min')

  callbacks_list = [checkpoint, earlystop]

In [ ]:
if 'DL' in featureCLF:
  print('saving model...')
  model.save( os.path.join(os.getcwd(), 'out', 'model', 'model_dl.h5') )

In [ ]:
runs = 1
folds = 5
batch = 450

if 'HOG' in featureCLF:
  features = ['Images', 'both']
else:
  features = ['Images', 'Spectral', 'both']
training = False
testing = True
if training:
  tipocurva = 'training'
else:
  tipocurva = 'testing'

skf = StratifiedKFold(n_splits=folds, shuffle = False, random_state = seed)

index_aux = np.arange( total_samples )
if 'DL' in featureCLF:
  y_aux = np.argmax( classes, axis = 1 )
else:
  y_aux = classes.copy()
results = True
resultsLC = True

for r in range(runs):

  k = 0
  index_aux, y_aux = shuffle( index_aux, y_aux, random_state = seed )
  for train_index, test_index in skf.split(index_aux, y_aux):
    print('run:',r+1,'; fold:',k+1,'...')
    
    train_index = train_index[:300]
    test_index = test_index[:100]
    

    if results:
      print('Clasification results...')
      if 'DL' in featureCLF:
        true, pred = get_results(index_aux[train_index], index_aux[test_index], featureCLF, None, data, classes, validation_split, Epochs, batch_size, callbacks_list)
        with open( os.path.join( os.getcwd(), 'out', 'Results', 'DL','results_'+'dl_'+featureCLF+'_r_'+str(r+1)+'_f_'+str(k+1)+'.pkl' ), 'wb' ) as a:
          pickle.dump( [true, pred], a, protocol = 2 )
      else:
        for feature in features:
          print(feature)
          true, pred_svm = get_results(index_aux[train_index], index_aux[test_index], featureCLF, feature, data, classes, None, None, None, None)
          with open( os.path.join( os.getcwd(), 'out', 'Results', 'SVM','results_svm_'+featureCLF+'_'+feature+'_r_'+str(r+1)+'_f_'+str(k+1)+'.pkl' ), 'wb' ) as a:
            pickle.dump( [true, pred_svm], a, protocol = 2 )

    if resultsLC:

      print('LC results...')
      if 'DL' in featureCLF:
        results_lc = get_lc_results( index_aux[train_index], index_aux[test_index], featureCLF, None, training, testing, data, classes, validation_split, Epochs, batch_size, callbacks_list, batch)
        with open( os.path.join( os.getcwd(), 'out', 'ResultsLC', 'DL', 'results_lc_dl_'+featureCLF+'_'+tipocurva+'_r_'+str(r+1)+'_f_'+str(k+1)+'.pkl' ), 'wb' ) as a:
          pickle.dump( results_lc, a, protocol = 2 )
      else:
        for feature in features:
          print(feature)
          results_lc_svm  = get_lc_results( index_aux[train_index], index_aux[test_index], featureCLF, feature, training, testing, data, classes, None, None, None, None, batch )
          with open( os.path.join( os.getcwd(), 'out', 'ResultsLC', 'SVM', 'results_lc_svm_'+featureCLF+'_'+feature+'_'+tipocurva+'_r_'+str(r+1)+'_f_'+str(k+1)+'.pkl' ), 'wb' ) as a:
            pickle.dump( results_lc_svm, a, protocol = 2 )
    k+=1